In [2]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.nddata import block_reduce
from acstools.findsat_mrt import TrailFinder
import matplotlib.pyplot as plt

def find_latest_dictionary_csv(folder_path):
    csv_files = [f for f in os.listdir(folder_path) if f.startswith('dictionary') and f.endswith('.csv')]
    if not csv_files:
        print("No dictionary CSV file found.")
        return None
    latest_csv = max(csv_files)
    return latest_csv

# Define the directory containing the FITS files
fits_dir = './Data/fits/'
# Define the directory to save filtered images
filtered_dir = './Data/fits_filtered9/'

# Create the filtered directory if it doesn't exist
os.makedirs(filtered_dir, exist_ok=True)

# Boolean flag to indicate whether to process only files with 'True' label in the CSV
process_only_labeled_true = True
label_column = 'process'  # Column in the CSV file that indicates if the file should be processed

# Get the latest CSV file
latest_csv = find_latest_dictionary_csv(fits_dir)

# Check if a CSV file was found
if latest_csv:
    # Load the latest CSV file
    csv_path = os.path.join(fits_dir, latest_csv)
    dictionary_df = pd.read_csv(csv_path)
    display(dictionary_df)

    # Filter the DataFrame if the flag is set to True
    if process_only_labeled_true:
        dictionary_df = dictionary_df[dictionary_df[label_column] == True]
    
    # Get a list of FITS files in the directory that match entries in the CSV
    fits_files = [f for f in os.listdir(fits_dir) if f.endswith('.fit') and f in dictionary_df['output'].values]
    
    # Loop over each FITS file that needs processing
    for fits_file in fits_files:
        # Construct the full path to the FITS file
        fits_path = os.path.join(fits_dir, fits_file)
        
        # Load the FITS image
        image = fits.getdata(fits_path, ext=0)
        
        # Subtract the median
        image = image - np.nanmedian(image)
        
        # Reduce the image size
        reduced_image = block_reduce(image, 1, func=np.nansum)
        
        # Apply the TrailFinder filter
        s = TrailFinder(reduced_image, processes=2, threshold=2, min_length=200, min_persistence=0.4, max_width=30)
        s.plot_image(overlay_mask=False)
        
        # Remove axis and title
        plt.axis('off')
        plt.title('')
        
        # Save the filtered image without white margins
        filtered_image_path = os.path.join(filtered_dir, f'{os.path.splitext(fits_file)[0]}.png')
        plt.savefig(filtered_image_path, bbox_inches='tight')
        plt.close()
        print(f"Filtered image saved: {filtered_image_path}")
        
        # Update the filename in the CSV file
        old_filename = os.path.splitext(fits_file)[0] + '.fit'
        new_filename = os.path.splitext(fits_file)[0] + '.png'
        dictionary_df['output'] = dictionary_df['output'].replace(old_filename, new_filename)

    # Save the updated CSV file
    updated_csv_path = os.path.join(filtered_dir, latest_csv)
    display(dictionary_df)
    dictionary_df.to_csv(updated_csv_path, index=False)
    print(f"Updated CSV file saved: {updated_csv_path}")
else:
    print("No CSV file found.")


,input,output,label
0,tic1.fit,tic1.fit,1
1,tic2.fit,tic2.fit,1
2,tic3.fit,tic3.fit,1
3,A012_ PSN_J12355230plus2755559_R_600s_ZWO ASI1...,tic4.fit,0
4,MPC_Cslash2022-QE78_B_180s_ZWO ASI1600MM-Cool_...,tic5.fit,0
5,tic6.fit,tic6.fit,1
6,A012_ PSN_J12355230plus2755559_R_600s_ZWO ASI1...,tic7.fit,0
7,MPC_12PslashPons-Brooks_B_180s_ZWO ASI1600MM-C...,tic8.fit,0
8,MPC_12PslashPons-Brooks_B_180s_ZWO ASI1600MM-C...,tic9.fit,0
9,MPC_12PslashPons-Brooks_R_120s_ZWO ASI1600MM-C...,tic10.fit,1


KeyError: 'process'